In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import feather
import re
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
import seaborn as sns
from seaborn import plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import train_test_split


pd.set_option('display.max_columns', None)
%matplotlib inline

In [ ]:
#Load DAta
df = feather.read_dataframe('NBA_Player_data_2017.feather')
standingsdf = feather.read_dataframe('df2017standings.feather')

In [ ]:
#Fix Rows
def fix_names(lineup, player):
    x = lineup.split('|')[1]
    split2 = x.split('. ')[1]
    splitj = player.split(' ')[1]
    if split2 == splitj:
        new_second = lineup.split('|')[0]
        new_first = lineup.split('|')[1]
        final_list = (new_first+' | '+ new_second)
        return final_list
    else:
        return lineup
    
df['new_lineup'] = df.apply(lambda row: fix_names(row['Lineup'],row['playername']), axis=1)

#Dop Column
del df['Lineup']

In [ ]:
df.MP = df.MP.map(lambda x: x.replace(':', '.'))

In [ ]:
# Fix Column names for machine learning
df[['FG','FGA', 'FG%', '3P', '3PA', '3P%', 'eFG%',
    'FT', 'FTA', 'FT%', 'PTS', 'ORB', 'ORB%', 'DRB', 'DRB%','TRB',
    'TRB%', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'MP']] = df[['FG','FGA', 'FG%',
                                                           '3P', '3PA', '3P%', 'eFG%', 'FT', 'FTA', 'FT%',
                                                           'PTS', 'ORB', 'ORB%', 'DRB', 'DRB%','TRB', 'TRB%', 'AST',
                                                           'STL', 'BLK', 'TOV', 'PF', 'MP']].apply(pd.to_numeric)

In [ ]:
# Weed out players who only played garbage minutes
df=df[df.MP > 100]

def prepare_merge_full():
    team_fullname_list = []
    for i in standingsdf.Team:
        team_fullname_list.append(i)
        team_fullname_list.sort()
    return team_fullname_list

def prepare_merge_abv():
    team_abv_list = []
    for i in df.Tm.unique():
        team_abv_list.append(i)
    return team_abv_list

team_dictionary = dict(zip(prepare_merge_abv(), prepare_merge_full()))
team_dictionary.get('CHI') and team_dictionary.update({'CHI': 'Chicago Bulls'})
team_dictionary.get('CHO') and team_dictionary.update({'CHO': 'Charlotte Hornets'})
team_dictionary.get('ATL') and team_dictionary.update({'ATL': 'Atlanta Hawks'})
team_dictionary.get('BOS') and team_dictionary.update({'BOS': 'Boston Celtics'})
team_dictionary.get('BRK') and team_dictionary.update({'BRK': 'Brooklyn Nets'})
team_dictionary.get('CLE') and team_dictionary.update({'CLE': 'Cleveland Cavaliers'})
team_dictionary.get('DAL') and team_dictionary.update({'DAL': 'Dallas Mavericks'})
team_dictionary.get('DEN') and team_dictionary.update({'DEN': 'Denver Nuggets'})
team_dictionary.get('DET') and team_dictionary.update({'DET': 'Detroit Pistons'})
team_dictionary.get('GSW') and team_dictionary.update({'GSW': 'Golden State Warriors'})
team_dictionary.get('HOU') and team_dictionary.update({'HOU': 'Houston Rockets'})
team_dictionary.get('IND') and team_dictionary.update({'IND': 'Indiana Pacers'})
team_dictionary.get('LAC') and team_dictionary.update({'LAC': 'Los Angeles Clippers'})
team_dictionary.get('LAL') and team_dictionary.update({'LAL': 'Los Angeles Lakers'})
team_dictionary.get('MEM') and team_dictionary.update({'MEM': 'Memphis Grizzlies'})
team_dictionary.get('MIA') and team_dictionary.update({'MIA': 'Miami Heat'})
team_dictionary.get('MIL') and team_dictionary.update({'MIL': 'Milwaukee Bucks'})
team_dictionary.get('MIN') and team_dictionary.update({'MIN': 'Minnesota Timberwolves'})
team_dictionary.get('NYK') and team_dictionary.update({'NYK': 'New York Knicks'})
team_dictionary.get('NOP') and team_dictionary.update({'NOP': 'New Orleans Pelicans'})
team_dictionary.get('PHI') and team_dictionary.update({'PHI': 'Philadelphia 76ers'})
team_dictionary.get('PHO') and team_dictionary.update({'PHO': 'Phoenix Suns'})
team_dictionary.get('POR') and team_dictionary.update({'POR': 'Portland Trailblazers'})
team_dictionary.get('SAC') and team_dictionary.update({'SAC': 'Sacramento Kings'})
team_dictionary.get('SAS') and team_dictionary.update({'SAS': 'San Antonio Spurs'})
team_dictionary.get('TOR') and team_dictionary.update({'TOR': 'Toronto Raptors'})
team_dictionary.get('UTA') and team_dictionary.update({'UTA': 'Utah Jazz'})
team_dictionary.get('WAS') and team_dictionary.update({'WAS': 'Washington Wizards'})
team_dictionary.get('OKC') and team_dictionary.update({'OKC': 'Oklahoma City Thunder'})
team_dictionary.get('ORL') and team_dictionary.update({'ORL': 'Orlando Magic'})


df['Team'] = df['Tm'].map(team_dictionary)

#Merge Standings with df
standingsdf = standingsdf[['Team', 'Overall']]
fulldf = pd.merge(df, standingsdf, on='Team')

In [ ]:
# Make Wins and Losses and Percentage columns
fulldf['Wins'], fulldf['Losses'] = fulldf['Overall'].str.split('-', 1).str

#Change to numeric and convert to total win percentage
fulldf[['Wins','Losses']] = fulldf[['Wins','Losses']].apply(pd.to_numeric)
fulldf['WinPerc'] = fulldf['Wins'] / (fulldf['Wins'] + fulldf['Losses'])

In [ ]:
# Doesn't like some column names -- Change (% and 3)
fulldf.rename(columns={'FG%':'FGPERC',
                       '3P':'THREEPT',
                       '3PA': 'THREEPTA',
                       '3P%':'THREEPTPERC',
                       'eFG%':'eFGPERC',
                       'FT%': 'FTPERC',
                      'ORB%' : 'ORBPERC',
                      'DRB%' : 'DRBPERC',
                      'TRB%' : 'TRBPERC'}, inplace=True)

In [ ]:
path = 'NBA_DATA_1.feather'
feather.write_dataframe(fulldf, path)